In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.md")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": retriver, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("Descrive Tom's character")

AIMessage(content='Tom Buchanan is described as a sturdy, straw-haired man in his thirties with a rather hard mouth and a supercilious manner. He has two shining, arrogant eyes that give him an aggressive appearance. He exudes a sense of power and dominance, with a body capable of enormous leverage. Tom is portrayed as someone who has achieved great physical accomplishments, particularly in football during his time at New Haven. He comes from an enormously wealthy family and has a freedom with money that was even a matter for reproach in college. Tom is depicted as someone who is fractious and has a touch of paternal contempt in his speaking voice. Despite his harsh demeanor, he seems to want others to like him with a harsh, defiant wistfulness of his own.')